In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path

%matplotlib inline


Bad key savefig.frameon in file /Users/bencifuentes/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /Users/bencifuentes/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /Users/bencifuentes/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout

In [3]:
df = pd.read_csv('Resources/data2.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df.head()

FileNotFoundError: [Errno 2] File b'Resources/data2.csv' does not exist: b'Resources/data2.csv'

In [ ]:
df = pd.read_csv

In [ ]:
 # Creating the X and y sets
X = df.drop(columns="positive_return")
y = df["positive_return"].values.astype("int")

In [ ]:
# Import the StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [ ]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=78)

In [ ]:
# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Model set-up
number_input_features = 4
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 5

In [ ]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Layer 3
model.add(Dense(units=hidden_nodes_layer3, activation="relu"))

# Layer 4 
model.add(Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
model.add(Dense(1, activation="sigmoid"))

In [ ]:
# Compile the model
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

In [ ]:
# Summarize the model
model.summary()

In [ ]:
# Training the model
batch_size = 1000
epochs = 500
model_training = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

In [ ]:
 # Plotting loss
loss_df = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": model_training.history["loss"],
        "Val": model_training.history["val_loss"],
    }
)
loss_df.set_index("Epoch", inplace=True)
loss_df.plot(title="Loss")

In [ ]:
# Plotting accuracy
accuracy_df = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": model_training.history["accuracy"],
        "Val": model_training.history["val_accuracy"],
    }
)
accuracy_df.set_index("Epoch", inplace=True)
accuracy_df.plot(title="Accuracy")

In [ ]:
 # Plotting AUC
auc_df = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": model_training.history["auc"],
        "Val": model_training.history["val_auc"],
    }
)
auc_df.set_index("Epoch", inplace=True)
auc_df.plot(title="AUC")

In [ ]:
# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

# Define metrics dictionary
metrics = {k: v for k, v in zip(model.metrics_names, scores)}

# Display evaluation metrics results
display(metrics)

In [ ]:
 # Define the confusion matrix data
cm_df = pd.DataFrame(
    {
        "Positive (1)": [f"TP={metrics['tp']}", f"FP={metrics['fn']}"],
        "Negative (0)": [f"FN={metrics['fp']}", f"TN={metrics['tn']}"],
    },
    index=["Positive(1)", "Negative(0)"],
)
cm_df.index.name = "Actual"
cm_df.columns.name = "Predicted"

# Show the confusion matrix
cm_df

In [ ]:
# Import the classification_report method from sklearn
from sklearn.metrics import classification_report

# Predict classes using testing data
y_predict_classes = model.predict_classes(X_test, batch_size=1000)

# Display classification report
print(classification_report(y_predict_classes, y_test))

In [ ]:
# Define the LSTM RNN model
model_2 = Sequential()

# Layer 1
model_2.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model_2.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Layer 3
model_2.add(Dense(units=hidden_nodes_layer3, activation="relu"))

# Layer 4 
model_2.add(Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
model_2.add(Dense(1, activation="sigmoid"))

In [ ]:
# Compile the model
model_2.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

In [ ]:
model_2.summary()

In [ ]:
# Training the model
batch_size = 1000
epochs = 2000
model_training_2 = model_2.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

In [ ]:
 # Plotting loss
loss_df_2 = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": model_training_2.history["loss"],
        "Val": model_training_2.history["val_loss"],
    }
)
loss_df_2.set_index("Epoch", inplace=True)
loss_df_2.plot(title="Loss")

In [ ]:
# Plotting accuracy
accuracy_df_2 = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": model_training_2.history["accuracy"],
        "Val": model_training_2.history["val_accuracy"],
    }
)
accuracy_df_2.set_index("Epoch", inplace=True)
accuracy_df_2.plot(title="Accuracy")

In [ ]:
 # Plotting AUC
auc_df_2 = pd.DataFrame(
    {
        "Epoch": range(1, epochs + 1),
        "Train": model_training_2.history["auc"],
        "Val": model_training_2.history["val_auc"],
    }
)
auc_df_2.set_index("Epoch", inplace=True)
auc_df_2.plot(title="AUC")

In [ ]:
scores_2 = model_2.evaluate(X_test, y_test, verbose=0)

# Define metrics dictionary
metrics_2 = {k: v for k, v in zip(model_2.metrics_names, scores_2)}

# Display evaluation metrics results
display(metrics_2)

In [ ]:
 # Define the confusion matrix data
cm_df_2 = pd.DataFrame(
    {
        "Positive (1)": [f"TP={metrics_2['tp']}", f"FP={metrics_2['fn']}"],
        "Negative (0)": [f"FN={metrics_2['fp']}", f"TN={metrics_2['tn']}"],
    },
    index=["Positive(1)", "Negative(0)"],
)
cm_df_2.index.name = "Actual"
cm_df_2.columns.name = "Predicted"

# Show the confusion matrix
cm_df_2

In [ ]:
# Import the classification_report method from sklearn
from sklearn.metrics import classification_report

# Predict classes using testing data
y_predict_classes_2 = model_2.predict_classes(X_test, batch_size=1000)

# Display classification report
print(classification_report(y_predict_classes_2, y_test))